## Saving a Cats v Dogs Model

This is a minimal example showing how to train a fastai model on Kaggle, and save it so you can use it in your app.

In [ ]:
# Make sure we've got the latest version of fastai:
!pip install -Uqq fastai

# Gradio for interface
!pip install gradio

First, import all the stuff we need from fastai:

In [6]:
from fastai.vision.all import *

Download and decompress our dataset, which is pictures of dogs and cats:

In [7]:
path = untar_data(URLs.PETS)/'images'

We need a way to label our images as dogs or cats. In this dataset, pictures of cats are given a filename that starts with a capital letter:

In [8]:
def is_cat(x): return x[0].isupper() 

Now we can create our `DataLoaders`:

In [9]:
dls = ImageDataLoaders.from_name_func('.',
    get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat,
    item_tfms=Resize(192))

... and train our model, a resnet18 (to keep it small and fast):

In [10]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,error_rate,time
0,0.196957,0.053650,0.016915,00:37


epoch,train_loss,valid_loss,error_rate,time
0,0.074636,0.069982,0.017591,00:30
1,0.044842,0.042084,0.012855,00:31
2,0.021600,0.021941,0.006766,00:30


Now we can export our trained `Learner`. This contains all the information needed to run the model:

In [11]:
learn.export('model.pkl')

Finally, open the Kaggle sidebar on the right if it's not already, and find the section marked "Output". Open the `/kaggle/working` folder, and you'll see `model.pkl`. Click on it, then click on the menu on the right that appears, and choose "Download". After a few seconds, your model will be downloaded to your computer, where you can then create your app that uses the model.

In [14]:
import gradio as gr
from fastai.vision.all import *
import numpy as np
from PIL.Image import Image as PILImageOriginal  # ✅ Correct class import

learn_inf = load_learner('model.pkl')

def classify_image(img):
    if isinstance(img, PILImageOriginal):
        img = PILImage.create(np.array(img))
    else:
        img = PILImage.create(img)

    pred, idx, probs = learn_inf.predict(img)
    labels = learn_inf.dls.vocab
    return {label: float(prob) for label, prob in zip(labels, probs)}


gr.Interface(fn=classify_image, 
             inputs=gr.Image(type="pil"), 
             outputs=gr.Label()).launch()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 29.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 KB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 KB 8.1 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5600 sha256=0d0abb9a24a3116efee05ba16eb9d3a5e9fe7a8da30805721463cd0ae1de6bca
  Stored in directory: /root/.cache/pip/wheels/81/c3/08/a4932aee56b934891932e4a1b189604f27d3d4e92b2ecd5ed4
Successfully built ffmpy
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.11.2
    Uninstalling Pygments-